In [2]:
#Importing Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests
import os
import json
import time
from scipy.stats import linregress
from api_keys import weather_api_key
# Import API key
from api_keys import g_key

#Access maps with unique API key

In [3]:
#Pulling Data from CSV
file = "archive/winemag-data-130k-v2.csv"
#Creating Base Dataframe
df = pd.read_csv(file, index_col=0)
df

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
Index,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser


In [ ]:
#Dropping usless columns
df = df.drop(columns= ['region_2', 'taster_twitter_handle'])
df

In [ ]:
#Reorganizing order of header
df = df[['winery','title','variety','country','province','region_1','price','points','taster_name','description', 'designation']]
df

In [ ]:
#Renaming/Cleaning headers 
df = df.rename(columns={'variety': 'Grape Type',
                       'region_1': 'Sub-Region',
                       'taster_name': 'Wine Critic',
                       'designation': 'Additional Designations',
                       'winery': 'Winery',
                       'title': 'Name of Wine',
                       })
df

In [ ]:
#Cleaning Headers
df.columns = df.columns.str.title()

In [ ]:
df

In [ ]:
#Dropping Nan values in important data metrics 
df = df.dropna(axis=0, how= 'any', subset= ['Price','Points','Wine Critic'])
df

In [ ]:
# Displaying boxplot/IQR of prices of wines by country
country = sns.boxplot(y="Country", x="Price", data=df, whis=np.inf)
#country = sns.stripplot(y="Country", x="Price", data=df, color=".3")
country = sns.set(rc={'figure.figsize':(20,80)}, font_scale= 1.5)

In [ ]:
#Describing the main data frame
df.describe()

In [ ]:
#Grouping Data by Country
country_group = df.groupby(by= 'Country')
#Finding mean of country data
country_df = country_group.mean()
country_df

In [ ]:
#Sorting Data by Price(Ascending)
price_df = country_df.sort_values(by= 'Price')
price_df

In [ ]:
#Visualizing average price of a bottle of reviewed wine per country
price_df["Price"].plot(kind="bar", facecolor="red", figsize=(15,5));

In [ ]:
#Examining Switzerland data
swiss_df = df[df['Country'] == 'Switzerland']
swiss_df

In [ ]:
#Boxplot of Switzerland to try and uncover any outliers
#add a box plot + stripchart
swiss = sns.boxplot(x="Price", y="Country", data=swiss_df, whis=np.inf)
#swiss = sns.stripplot(x="Price", y="Country", data=swiss_df, color=".3")

In [ ]:
#Describing Swiss Data
swiss_df.describe()

In [ ]:
#Sorting Data by Points(Ascending)
points_df = country_df.sort_values(by= 'Points')
points_df

In [ ]:
#Plotting bar chart showing avg points given to a btl reviewed wine in each country
points_df["Points"].plot(kind="bar", facecolor="blue", figsize=(15,5));

In [ ]:
#Adding new column to uncover the average $ it costs consumers per point rating in each country
country_df['Average Price per Point ($)'] = round(country_df['Price']/country_df['Points'],2)
country_df

In [ ]:
#Sorting data by Avg Price per point
ppp_df = country_df.sort_values(by= 'Average Price per Point ($)')
ppp_df

In [ ]:
#Plotting Avg PPP data
ppp_df["Average Price per Point ($)"].plot(kind="bar", facecolor="blue", figsize=(15,5));

In [ ]:
#Displaying how many wines were reviewed per country
dfg = df.groupby(by= 'Country').count()
dfg = dfg[['Wine Critic']]
dfg = dfg.sort_values(by= 'Wine Critic')
dfg

In [ ]:
#Plotting in bar chart the amount of reviews per country
dfg['Wine Critic'].plot(kind='bar', facecolor='green', figsize=(15,5))

In [ ]:
#Grouping Data by Wine Critic
critic_group = df.groupby(by= 'Wine Critic')

critic_df = critic_group.count()
critic_df

In [ ]:
#Counting how many reviews each critic has done
crit_df = critic_df[['Winery']]
crit_df

In [ ]:
#Sorting data in Ascending order
crit_sort = crit_df.sort_values(by= 'Winery')
crit_sort

In [ ]:
#Plotting bar chart to show how many reviews done per critic
crit_sort['Winery'].plot(kind='bar', facecolor='purple', figsize=(15,5))

In [ ]:
#Finding the mean for Price and Points in a Wine Crtic Groupby
crit_mean = critic_group.mean()
crit_mean

In [ ]:
#Sorting Data by Points in Ascending
crit_points = crit_mean.sort_values(by= 'Points')
crit_points

In [ ]:
#Plotting mean of points by critic 
crit_points["Points"].plot(kind="bar", facecolor="blue", figsize=(15,5));

In [ ]:
#Sorting wine critic data by price in ascending
crit_price = crit_mean.sort_values(by= 'Price')
crit_price

In [ ]:
#Plotting bar chart to show avg price of btl reviewed by each critic
crit_price["Price"].plot(kind="bar", facecolor="blue", figsize=(15,5));

In [ ]:
#Creating new column calculating the avergae price of the wine per point given by each critic
crit_mean['Average Price per Point ($)'] = round(crit_mean['Price']/crit_mean['Points'],2)
crit_mean

In [ ]:
#Sorting the new average in ascending
crit_avg = crit_mean.sort_values(by= 'Average Price per Point ($)')
crit_avg

In [ ]:
#Plotting bar chart to show the new sorted data
crit_avg["Average Price per Point ($)"].plot(kind="bar", facecolor="blue", figsize=(15,5));

In [ ]:
# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)



url = f"https://maps.googleapis.com/maps/api/geocode/json?"

params = {'key': g_key}

# location_response = requests.get(url = url, params = params)
# location_data = location_response.json()


# define count of pass throughs
count_one = 0
set_one = 1

In [ ]:
country_list =[]
province_list = []

for province in df['Province']:
    province_list.append(province)
    

for country in df['Country']:
    country_list.append(country)

location_list = []
skipped = []

# for location in range(len(country_list)):
#     print(location)
#     try:
#         params['address'] = f"{country_list[location]},{province_list[location]}"
#         print(params['address'])
#         location_response = requests.get(url = url, params = params)
#         location_data = location_response.json()
#         location = {}
#         location['Latitude'] = location_data['results'][0]['geometry']['location']['lat']
#         location['Longitude'] = location_data['results'][0]['geometry']['location']['lng']
#         location_list.append(location)
        
#     except:
#         location['Latitude'] = np.nan
#         location['Longitude'] = np.nan
#         location_list.append(location)
#         print('Skipped')
#         continue
        
#     if location == 10:
#         break
        
        



In [ ]:
# df_location = pd.DataFrame(location_list)
df_location = pd.read_csv('archive/location_data.csv', index_col=0)
df_location

In [ ]:
df_location['Country'] = country_list
df_location['Province'] = province_list
df_location

In [ ]:
prov_grp = df_location.groupby(by='Province').mean()
prov_grp

In [ ]:
index = prov_grp.index
index

In [ ]:
len(prov_grp)

In [ ]:
from api_keys import weather_api_key
# Initialize values for for loop - 
# counter to keep track of records for print statement and 
count = 0
# empty list to store weather data to make into
province_weather = []

url = "http://api.openweathermap.org/data/2.5/weather?"
params = {'appid': weather_api_key,
         'units': 'metric'}

print(f'Beginning Data Retrieval')
print('-----------------------------')

# for loop to iterate through cities selected
for province in index:
    
    print(f'Processing Record {count} | {province}')
    
    if count > 415:
        break
    else:
        # Set URL paramater for the location
        print(count)
        count += 1
        params['lat'] = prov_grp.iloc[count, 0]
        params['lon'] = prov_grp.iloc[count, 1]

        # Exception handling - if city is not in OpenWeather API skip
        try:
            # Build query URL
            weather_response = requests.get(url = url, params = params)
            weather_data = weather_response.json()

            # Create dictionary for data from API call
            weather = {}

            # Create key's and assign data to it based on API call
            weather['Province'] = province
            weather['Description'] = weather_data['weather'][0]['description']
            weather['Lat'] = weather_data['coord']['lat']
            weather['Lng'] = weather_data['coord']['lon']
            weather['Temp'] = weather_data['main']['temp']
            weather['Humidity'] = weather_data['main']['humidity']
            weather['Cloudiness'] = weather_data['clouds']['all']
            weather['Wind Speed'] = weather_data['wind']['speed']
            weather['Country'] = weather_data['sys']['country']
            weather['Date'] = weather_data['dt']
            # Add dictionary to end of list
            province_weather.append(weather)
        except:
            print(f'Province not found. Skipping...')
            continue

print('-----------------------------')
print('Data Retrieval Complete') 
print('-----------------------------')

In [ ]:
province_weather = pd.DataFrame(province_weather)
province_weather

In [ ]:
weather_merge = pd.merge(df, province_weather, on=['Province'], how='left')

In [ ]:
weather_merge

In [ ]:
weather_clean = weather_merge.dropna(axis=0, how= 'any', subset= ['Temp', 'Lat', 'Country_y'])
weather_clean.describe()
weather_clean.nunique()

In [ ]:
# Lat vs temp
from matplotlib.pyplot import figure
figure(figsize=(8,10), dpi=80)
x_values = weather_clean['Lat']
y_values = weather_clean['Temp']
plt.scatter(x_values,y_values, edgecolors="black")
plt.title('Winery Latitude vs. Temperature (10/03/21)')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.grid(b=True, axis='both')
plt.tight_layout()

# plt.savefig("../images/lat_temp.png")
plt.show();

In [ ]:
# Create bins for different Hemispheres
bins = [-90, -.1, 90]

group_names = ['Southern Hemisphere', 'Northern Hemisphere']

weather_clean["Hemisphere"] = pd.cut(weather_clean['Lat'], bins, 
                                  labels=group_names, include_lowest=True)

#Filter by the bins
north_df = weather_clean[weather_clean['Hemisphere']=="Northern Hemisphere"]
south_df = weather_clean[weather_clean['Hemisphere']=="Southern Hemisphere"]

In [ ]:
x_values = north_df['Lat']
y_values = north_df['Temp']
figure(figsize=(8,10), dpi=80)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Winery Latitude')
plt.ylabel('Max Temp (F)')
plt.title('Northern Winery Latitude vs. Temperature (10/03/21)')
print(f"The r-value is: {rvalue}")
# plt.savefig("images/north_lat_maxtemp_regression.png")
plt.tight_layout()
plt.show()

In [ ]:
x_values = south_df['Lat']
y_values = south_df['Temp']
figure(figsize=(8,8), dpi=80)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-50,80),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Max Temp (F)')
plt.title('Southern Winery Latitude vs. Temperature (10/03/21)')
print(f"The r-value is: {rvalue}")
# plt.savefig("images/south_lat_maxtemp_regression.png")
plt.tight_layout()
plt.show()

In [ ]:
# Lat vs temp
from matplotlib.pyplot import figure``
figure(figsize=(8,10), dpi=80)
x_values = weather_clean['Lat']
y_values = weather_clean['Price']
plt.scatter(x_values,y_values, edgecolors="black")
plt.title('Winery Latitude vs. Price (10/03/21)')
plt.xlabel('Latitude')
plt.ylabel('Price ($ USD)')
plt.grid(b=True, axis='both')
plt.tight_layout()

# plt.savefig("../images/lat_temp.png")
plt.show();

In [ ]:
x_values = north_df['Lat']
y_values = north_df['Price']
figure(figsize=(8,10), dpi=80)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(10,40),fontsize=15,color="red")
plt.xlabel('Winery Latitude')
plt.ylabel('Price ($ USD)')
plt.title('Northern Winery Latitude vs. Price (10/03/21)')
print(f"The r-value is: {rvalue}")
# plt.savefig("images/north_lat_maxtemp_regression.png")
plt.tight_layout()
plt.show()

In [ ]:
x_values = south_df['Lat']
y_values = south_df['Price']
figure(figsize=(8,8), dpi=80)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(-50,80),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Price ($ USD)')
plt.title('Southern Winery Latitude vs. Price (10/03/21)')
print(f"The r-value is: {rvalue}")
# plt.savefig("images/south_lat_maxtemp_regression.png")
plt.tight_layout()
plt.show()

In [ ]:
df_varietal = df.groupby(by='Grape Type')
df_var = df_varietal.count()
df_sorted = df_var.sort_values(by='Winery', ascending=False)
df25 = df_sorted.head(25)
top_grape = df25.index
tg = pd.DataFrame(top_grape)
tg_mean = df.groupby(by='Grape Type').mean()
tg_merge = pd.merge(tg, tg_mean, how='left', right_index=True, left_on='Grape Type')
tg_mean
tg_merge


In [ ]:
reindexed = tg_merge.set_index('Grape Type')
reindexed